# Peer-graded Assignment: Final Assigment. Fast food restaurant placement in the city of Toronto

**Libraries Load**

In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
!conda install -c conda-forge folium=0.5.0 --yes
import folium 
!pip install beautifulsoup4
from bs4 import BeautifulSoup

import requests
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
!pip install lxml
import lxml

**Data loading and DF creation**

In [14]:
List_url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969"
source = requests.get(List_url).text
soup = BeautifulSoup(source, 'lxml')
table=soup.find('table')
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

**Cleaning Up**

In [15]:
df=df[df['Borough']!='Not assigned']

In [16]:
df_join=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df_join=df_join.reset_index(drop=False)
df_join.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

**Combine Neihborhoods, Rename Columns, Drop Duplicates, Merge and Show**

In [17]:
df.drop_duplicates(inplace=True)
df.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df.head(5)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**Shape of Dataframe**

In [18]:
df.shape

(103, 3)

# Getting Postal Code and Coords

**Get Postal Code Lat/Lng**

In [19]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [20]:
df_coor=pd.read_csv('https://cocl.us/Geospatial_data')

**Check if values are loaded correctly**

In [22]:
df_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Merge Dataframes**

In [25]:
df_coor.rename(columns={'Postal Code':'Postalcode'},inplace=True)
df_final = pd.merge(df_coor, df , on='Postalcode')

In [26]:
df_final = df_final[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [27]:
df_final.head(12) ## I limited it to show from 0 to 11 to match the assignment picture

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Get Toronto Venues

In [28]:
Toronto_df =df_final[df_final['Borough'].str.contains("Toronto")]

In [29]:
##Foursquare Variables

CLIENT_ID = 'FRNIXSF0QDYZEZIO4ELQR5CT2DCFJW2BKVYVO3OYRIANJE5D'
CLIENT_SECRET = '1YR5OXY2RO1QOAQYNE0C23ZIRKXUNZ50KMKMI3UMKXSXLCGA'
VERSION = '20180605'

In [30]:
## Same analysis as NY one

def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
## Get the venues
Venues_Toronto = getNearbyVenues(names=Toronto_df['Neighborhood'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
Runn

**DF check and grouping**

In [32]:
Venues_Toronto.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [33]:
#Grouping

Venues_Toronto.groupby('Neighborhood').count()

#Hot Encoding

Toronto_Hot = pd.get_dummies(Venues_Toronto[['Venue Category']], prefix="", prefix_sep="")
Toronto_Hot.drop(['Neighborhood'],axis=1,inplace=True) 
Toronto_Hot.insert(loc=0, column='Neighborhood', value=Venues_Toronto['Neighborhood'] )
Toronto_Hot.shape

(1613, 234)

In [34]:
#Final touchs
Toronto_Normalized = Toronto_Hot.groupby('Neighborhood').mean().reset_index()
Toronto_Normalized.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.016667,0.000000,0.0,0.0,0.0000,0.0,0.0,0.016667,0.050000,0.000000,0.000000,0.0,0.016667,0.016667,0.0,0.033333,0.0,0.0,0.016667,0.0,0.000000,0.000000,0.0,0.016667,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.0000,0.016667,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000,0.016667,0.05,0.083333,0.0,0.0,0.0,0.0,0.0,0.016667,0.000000,0.016667,0.000000,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,0.016667,0.0,0.016667,0.000000,0.0,0.0,0.0,0.000000,0.0,0.016667,0.0,0.0,0.0,0.0,0.000000,0.033333,0.0000,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,0.0,0.016667,0.016667,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0000,0.0,0.00000

In [35]:
# Sort top venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto_Final = pd.DataFrame(columns=columns)
Toronto_Final['Neighborhood'] = Toronto_Normalized['Neighborhood']

for ind in np.arange(Toronto_Normalized.shape[0]):
    Toronto_Final.iloc[ind, 1:] = return_most_common_venues(Toronto_Normalized.iloc[ind, :], num_top_venues)

Toronto_Final.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Cheese Shop,Pharmacy,Restaurant,Farmers Market,Beer Bar,Shopping Mall
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Coffee Shop,Breakfast Spot,Burrito Place,Convenience Store,Stadium,Restaurant,Italian Restaurant,Bar
2,"Business reply mail Processing Centre, South C...",Pizza Place,Garden,Restaurant,Butcher,Burrito Place,Skate Park,Brewery,Smoke Shop,Park,Light Rail Station
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Plane
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Salad Place,Thai Restaurant,Yoga Studio


**Clustering time**

In [37]:
# set number of clusters
kclusters = 5

Toronto_Cluster = Toronto_Normalized.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_Cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [38]:
# Cluster Labels and merge

Toronto_Final.insert(0, 'Cluster Labels', kmeans.labels_)
df_clusters = Toronto_df
df_clusters = df_clusters.join(Toronto_Final.set_index('Neighborhood'), on='Neighborhood')
df_clusters.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Health Food Store,Trail,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Bookstore,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Juice Bar
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Pizza Place,Fast Food Restaurant,Liquor Store,Board Shop,Italian Restaurant,Restaurant,Fish & Chips Shop,Food & Drink Shop,Sushi Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Café,Bakery,Gastropub,American Restaurant,Brewery,Yoga Studio,Italian Restaurant,Fish Market,Coworking Space
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Yoga Studio,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


In [39]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates for Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates for Toronto are 43.6534817, -79.3839347.


**Create Map**

In [40]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_clusters['Latitude'], df_clusters['Longitude'], df_clusters['Neighborhood'], df_clusters['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6).add_to(map_clusters)
       
map_clusters

**Let's analyze the clusters!! (Coments Below each)**

In [41]:
df_clusters.loc[df_clusters['Cluster Labels'] == 0, df_clusters.columns[[1] + list(range(5, df_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Pub,Health Food Store,Trail,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Bookstore,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Juice Bar
42,East Toronto,0,Park,Pizza Place,Fast Food Restaurant,Liquor Store,Board Shop,Italian Restaurant,Restaurant,Fish & Chips Shop,Food & Drink Shop,Sushi Restaurant
43,East Toronto,0,Coffee Shop,Café,Bakery,Gastropub,American Restaurant,Brewery,Yoga Studio,Italian Restaurant,Fish Market,Coworking Space
45,Central Toronto,0,Pizza Place,Hotel,Gym / Fitness Center,Breakfast Spot,Sandwich Place,Park,Department Store,Food & Drink Shop,Dog Run,Doner Restaurant
46,Central Toronto,0,Coffee Shop,Clothing Store,Yoga Studio,Furniture / Home Store,Restaurant,Diner,Salon / Barbershop,Ice Cream Shop,Mexican Restaurant,Chinese Restaurant
47,Central Toronto,0,Pizza Place,Dessert Shop,Sandwich Place,Gym,Café,Coffee Shop,Sushi Restaurant,Italian Restaurant,Toy / Game Store,Diner
49,Central Toronto,0,Coffee Shop,Pizza Place,Pub,Café,Restaurant,Bank,Supermarket,Sushi Restaurant,Bagel Shop,Liquor Store
51,Downtown Toronto,0,Coffee Shop,Café,Gourmet Shop,Restaurant,Pub,Italian Restaurant,Bakery,Pizza Place,Park,Butcher
52,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Pub,Men's Store,Mediterranean Restaurant,Hotel


**This is the cluster we are looking for to analyze and extrapolate it's data. There are a lot of venues but not many Fast Food Restaurants. This spot has room for another Fast Food venue!**

In [42]:
df_clusters.loc[df_clusters['Cluster Labels'] == 1, df_clusters.columns[[1] + list(range(5, df_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Lawyer,Park,Trail,Summer Camp,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
50,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
64,Central Toronto,1,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


**Not relevant to our client**

In [43]:
df_clusters.loc[df_clusters['Cluster Labels'] == 2, df_clusters.columns[[1] + list(range(5, df_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,2,Garden,Health & Beauty Service,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


**Not relevant to our client**

In [44]:
df_clusters.loc[df_clusters['Cluster Labels'] == 3, df_clusters.columns[[1] + list(range(5, df_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
81,Toronto/York,3,Grocery Store,Convenience Store,Brewery,Yoga Studio,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


**Not relevant to our client**

In [45]:
df_clusters.loc[df_clusters['Cluster Labels'] == 4, df_clusters.columns[[1] + list(range(5, df_clusters.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4,Park,Swim School,Bus Line,Yoga Studio,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


**Not relevant to our client**

# Now that we have the venue data and we decided a zone to open the fast food restaurant, let's check the demographic in there by analyzing population density!

### Dataset reading

In [71]:
url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "9308a7e1-3781-45fd-95c7-582b2030f2c1"}
package = requests.get(url, params = params).json()
print(package["result"])


for idx, resource in enumerate(package["result"]["resources"]):
    if resource["datastore_active"]:
        url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/datastore_search"
        p = { "id": resource["id"] }
        data = requests.get(url, params = p).json()
        df = pd.DataFrame(data["result"]["records"])
        break
df.head()

{'license_title': 'Open Government Licence – Toronto', 'owner_unit': None, 'relationships_as_object': [], 'topics': 'Community services', 'owner_email': 'spar@toronto.ca', 'excerpt': 'Information on alternative schools and adult learning centres. Also includes school boards and colleges and universities. ', 'private': False, 'owner_division': 'Social Development, Finance & Administration', 'num_tags': 7, 'id': '9308a7e1-3781-45fd-95c7-582b2030f2c1', 'metadata_created': '2020-03-04T18:15:00.345658', 'refresh_rate': 'As available', 'title': 'Wellbeing Youth - Alternative Adult Education', 'license_url': 'https://open.toronto.ca/open-data-license/', 'state': 'active', 'information_url': 'https://www.toronto.ca/city-government/data-research-maps/neighbourhoods-communities/wellbeing-toronto/', 'license_id': 'open-government-licence-toronto', 'type': 'dataset', 'resources': [{'cache_last_updated': None, 'package_id': '9308a7e1-3781-45fd-95c7-582b2030f2c1', 'datastore_active': True, 'id': 'a8

,_id,OBJECTID,AGENCY_NAME,ORGANIZATION_ADDRESS,NEIGHBOURHOOD,OFFICE_PHONE,EMAIL,WEBSITE,ELIGIBILITY,DESCRIPTION_SERVICE,APPLICATION,LANGUAGES,ACCESSIBILITY,HOURS,LEGAL_STATUS,DATE_UPDATED,ADDRESS_POINT_ID,X,Y,LONGITUDE,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE,geometry
0,1,262,Joint Training and Apprenticeship Committee,"936 Warden Ave, Toronto, ON M1L 4C9","Wexford-Maryvale, 119",416-759-9351,jtacoffice@ualocal46.org,"<a href=""http://www.ualocal46.org"" target=""_bl...",United Association Local 46 member * or enroll...,Apprenticeship programs in plumbing and steamf...,Course description and application form availa...,English,Wheelchair accessible building including main ...,Mon-Fri 8 am-4 pm,(50) Non Profit,2015-07-22T04:00:00,395765,None,None,None,None,936 Warden Ave,Scarborough,M1L 4C9,"{""type"": ""Point"", ""coordinates"": [-79.29001478..."
1,2,263,Ontario Industrial and Finishing Skills Centre,"130 Toro Rd, Unit C, Toronto, ON M3J 3M9","York University Heights, 27",416-635-7300,info@oifsc.com,"<a href=""http://www.oifsc.com"" target=""_blank""...","Minimum Grade 10 English, math and science or ...",Apprenticeship programs for painter decorators...,Apply through local union or employer,English,Wheelchair accessible -- call for details,Mon-Fri 8:30 am-4:30 pm,(50) Non Profit,2015-07-27T04:00:00,9171851,None,None,None,None,130 Toro Rd,North York,M3J 3M9,"{""type"": ""Point"", ""coordinates"": [-79.48321492..."
2,3,264,Toronto Ironworkers Local 721,"909 Kipling Ave, Toronto, ON M8Z 5H3","Islington-City Centre West, 14",416-232-1046,ironworkerslocal721@rogers.com,"<a href=""http://www.iw721.org"" target=""_blank""...",Must have Grade 10,Apprenticeship programs for ironworkers and ro...,Call for information,English,Wheelchair accessible building including main ...,Mon-Fri 7:30 am-3 pm,(50) Non Profit,2015-08-25T04:00:00,1011523,None,None,None,None,909 Kipling Ave,Etobicoke,M8Z 5H3,"{""type"": ""Point"", ""coordinates"": [-79.53300702..."
3,4,265,Toronto Catholic District School Board,"Catholic Education Centre, 80 Sheppard Ave E, ...","Willowdale East, 51",416-222-8282 ext 5314 ; Admissions 416-222-828...,webmaster@tcdsb.org,"<a href=""http://www.tcdsb.org"" target=""_blank""...",Elementary level -- Roman Catholic children an...,Full academic program in a Roman Catholic envi...,Contact nearest Catholic school for appointmen...,English ; French - (school programs) ; Interpr...,Wheelchair accessible building including main ...,Mon-Fri 8:30 am-4:45 pm * after hours -- answe...,(50) Non Profit,2015-07-08T04:00:00,5200663,None,None,None,None,80 Sheppard Ave E,North York,M2N 6E8,"{""type"": ""Point"", ""coordinates"": [-79.40809597..."
4,5,266,Academy of Computer and Employment Skills,"55 Eglinton Ave E, Ste 703, Toronto, ON M4...","Mount Pleasant West, 104",416-703-7770 ext 206,marina.n@workingskillscentre.com,"<a href=""http://www.workingskillscentre.com/di...",Canadian Language Benchmark Level 4 to Level 7...,Registered private career college * vocational...,Information sessions Thursday 9 am * academic ...,English,Wheelchair accessible building including main ...,Mon-Fri 8:30 am-6 pm * some evenings and Satur...,(50) Non Profit ; (51) Registered Charity,2013-12-06T05:00:00,800592,None,None,None,None,55 Eglinton Ave E,former Toronto,M4P 1G8,"{""type"": ""Point"", ""coordinates"": [-79.39607782..."


## Cleaning up the data

In [72]:
 df.drop(labels=["_id","OBJECTID","ORGANIZATION_ADDRESS","NEIGHBOURHOOD","OFFICE_PHONE","EMAIL","WEBSITE","ELIGIBILITY","DESCRIPTION_SERVICE","APPLICATION","LANGUAGES","ACCESSIBILITY","HOURS","LEGAL_STATUS","DATE_UPDATED","ADDRESS_POINT_ID","X","Y","ADDRESS_FULL","MUNICIPALITY","geometry"], axis=1, index=None, columns=None, level=None, inplace=True)

In [74]:
df

,AGENCY_NAME,LONGITUDE,LATITUDE,POSTAL_CODE
0,Joint Training and Apprenticeship Committee,None,None,M1L 4C9
1,Ontario Industrial and Finishing Skills Centre,None,None,M3J 3M9
2,Toronto Ironworkers Local 721,None,None,M8Z 5H3
3,Toronto Catholic District School Board,None,None,M2N 6E8
4,Academy of Computer and Employment Skills,None,None,M4P 1G8
5,Redemption Reintegration Services,None,None,M1P 3B9
6,Conseil scolaire Viamonde,None,None,M6L 2K5
7,Toronto District School Board,None,None,M2N 5N8
8,Humber College,None,None,M9W 5L7
9,Seneca College,None,None,M2J 2X5


In [75]:
df.rename(columns={'POSTAL_CODE':'Postalcode'},inplace=True)
df.rename(columns={'AGENCY_NAME':'University'},inplace=True)
df.rename(columns={'LONGITUDE':'Longitude'},inplace=True)
df.rename(columns={'LATITUDE':'Latitude'},inplace=True)

In [76]:
df['Postalcode'] = df['Postalcode'].str[:3]

In [77]:
df

,University,Longitude,Latitude,Postalcode
0,Joint Training and Apprenticeship Committee,None,None,M1L
1,Ontario Industrial and Finishing Skills Centre,None,None,M3J
2,Toronto Ironworkers Local 721,None,None,M8Z
3,Toronto Catholic District School Board,None,None,M2N
4,Academy of Computer and Employment Skills,None,None,M4P
5,Redemption Reintegration Services,None,None,M1P
6,Conseil scolaire Viamonde,None,None,M6L
7,Toronto District School Board,None,None,M2N
8,Humber College,None,None,M9W
9,Seneca College,None,None,M2J


In [78]:
df_u = pd.merge(df_coor, df , on='Postalcode')

In [79]:
df_u

,Postalcode,Latitude_x,Longitude_x,University,Longitude_y,Latitude_y
0,M1L,43.711112,-79.284577,Joint Training and Apprenticeship Committee,None,None
1,M1P,43.757410,-79.273304,Redemption Reintegration Services,None,None
2,M2J,43.778517,-79.346556,Seneca College,None,None
3,M2M,43.789053,-79.408493,Conseil scolaire de district catholique Centre...,None,None
4,M2N,43.770120,-79.408493,Toronto Catholic District School Board,None,None
5,M2N,43.770120,-79.408493,Toronto District School Board,None,None
6,M3J,43.767980,-79.487262,Ontario Industrial and Finishing Skills Centre,None,None
7,M3J,43.767980,-79.487262,York University,None,None
8,M3M,43.728496,-79.495697,LIUNA Local 183 Training Centre,None,None
9,M4K,43.679557,-79.352188,Ambassador Program,None,None


In [82]:
 df_u.drop(labels=["Longitude_y","Latitude_y"], axis=1, index=None, columns=None, level=None, inplace=True)

In [83]:
df_u

,Postalcode,Latitude_x,Longitude_x,University
0,M1L,43.711112,-79.284577,Joint Training and Apprenticeship Committee
1,M1P,43.757410,-79.273304,Redemption Reintegration Services
2,M2J,43.778517,-79.346556,Seneca College
3,M2M,43.789053,-79.408493,Conseil scolaire de district catholique Centre...
4,M2N,43.770120,-79.408493,Toronto Catholic District School Board
5,M2N,43.770120,-79.408493,Toronto District School Board
6,M3J,43.767980,-79.487262,Ontario Industrial and Finishing Skills Centre
7,M3J,43.767980,-79.487262,York University
8,M3M,43.728496,-79.495697,LIUNA Local 183 Training Centre
9,M4K,43.679557,-79.352188,Ambassador Program


In [84]:
df_u.rename(columns={'Latitude_x':'Latitude'},inplace=True)
df_u.rename(columns={'Longitude_x':'Longitude'},inplace=True)

In [85]:
df_u

,Postalcode,Latitude,Longitude,University
0,M1L,43.711112,-79.284577,Joint Training and Apprenticeship Committee
1,M1P,43.757410,-79.273304,Redemption Reintegration Services
2,M2J,43.778517,-79.346556,Seneca College
3,M2M,43.789053,-79.408493,Conseil scolaire de district catholique Centre...
4,M2N,43.770120,-79.408493,Toronto Catholic District School Board
5,M2N,43.770120,-79.408493,Toronto District School Board
6,M3J,43.767980,-79.487262,Ontario Industrial and Finishing Skills Centre
7,M3J,43.767980,-79.487262,York University
8,M3M,43.728496,-79.495697,LIUNA Local 183 Training Centre
9,M4K,43.679557,-79.352188,Ambassador Program


In [86]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates for Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates for Toronto are 43.6534817, -79.3839347.


In [101]:
Toronto_Map = folium.Map(location=[latitude, longitude], zoom_start=10)
Toronto_Map

In [102]:
Universities = folium.map.FeatureGroup()

for lat, lng, in zip(df_u.Latitude, df_u.Longitude):
    Universities.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=15,
            color='grey',
            fill=True,
            fill_color='orange',
            fill_opacity=0.6
        )
    )

# add incidents to map
Toronto_Map.add_child(Universities)

## Map insertion

In [103]:
Venues = folium.map.FeatureGroup()

for lat, lng, in zip(df_clusters.Latitude, df_clusters.Longitude):
    Venues.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='red',
            fill=True,
            fill_color='grey',
            fill_opacity=0.6
        )
    )


Toronto_Map.add_child(Venues)

## Thanks for evaluating!